In [1]:
import sys

import numpy as np
import pandas as pd
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params, gaussian_kernel
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import unet_builder as bare_model_builder

PWD = '../../..'
sys.path.append(PWD)

In [2]:
PROJECT_NAME = 'unet'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 4096
MAX_TRIALS = 30
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train_original, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train_original, y_val_original = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22134, 24), (17707, 24), (4427, 24))

In [4]:
y_base_train = np.array([gaussian_kernel(y) for y in y_base_train_original])
y_train = np.array([gaussian_kernel(y) for y in y_train_original])
y_val = np.array([gaussian_kernel(y) for y in y_val_original])

y_base_train.shape, y_train.shape, y_val.shape

((22134, 24), (17707, 24), (4427, 24))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_unet_depth = hp.Int("unet_depth", min_value=0, max_value=3, step=1, default=2)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8, 16], default=4)
    hp_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_unet_depth, hp_n_conv_layers, hp_filters_mult, hp_spatial_dropout,
                               hp_batch_normalization, hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 24, 1)        0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 24, 32)       96          ['reshape[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 12, 32)       0           ['conv1d[0][0]']                 
                                                                                              

In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1039969
========== Model 1 ==========
{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.2, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 259169
========== Model 2 ==========
{'unet_depth': 2, 'n_conv_layers': 1, 'conv_filters_mult': 16, 'conv_spatial_dropout': 0.2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 990305
========== Model 3 ==========
{'unet_depth': 1, 'n_conv_layers': 3, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.2, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 10081
========== Model 4 ==========
{'u

# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1039969
Got score: 13.2224 (13.2275, 13.2173)
Got score: 12.9089 (12.8842, 12.9336)
Got score: 12.7480 (12.7096, 12.7864)
Got score: 13.0311 (13.1519, 12.9102)
Got score: 12.8546 (12.8949, 12.8144)


{'unet_depth': 3, 'n_conv_layers': 1, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.2, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 259169
Got score: 13.8629 (13.5909, 14.1350)
Got score: 13.1407 (13.0657, 13.2156)
Got score: 12.9914 (12.9259, 13.0569)
Got score: 13.4540 (13.2463, 13.6618)
Got score: 12.9820 (13.0405, 12.9235)


{'unet_depth': 2, 'n_conv_layers': 1, 'conv_filters_mult': 16, 'conv_spatial_dropout': 0.2, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 990305
Got score: 42.0939 (70.6196, 13.5682)
Got score: 38.2024 (13.2497, 63.1551)
Got score: 38.5598 (13.2191, 63.9005)
Got score: 37.0165 (60.6452, 13.3878)
Got score: 13.4724 (13.3441, 13.6007)


{'unet_depth': 1, 'n_conv_layers': 3, 'conv_filters_mult': 2, 'conv_spatial_dropout': 0.2, 'batch_normalization': False, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 10081
Got score: 13.9847 (14.2364, 13.7329)
Got score: 13.7618 (14.0738, 13.4498)
Got score: 13.5153 (13.6299, 13.4008)
Got score: 16.1081 (18.5087, 13.7075)
Got score: 14.0842 (13.7534, 14.4149)


{'unet_depth': 2, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': True}
Number of parameters: 120641
Got score: 13.9306 (14.0657, 13.7954)
Got score: 13.3121 (13.1907, 13.4335)
Got score: 13.3225 (13.4851, 13.1599)
Got score: 13.3963 (13.4430, 13.3497)
Got score: 13.0962 (13.0977, 13.0946)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,12.95,0.16,1039969
1,13.29,0.34,259169
2,33.87,10.34,990305
3,14.29,0.93,10081
4,13.41,0.28,120641
